In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load the CSV files
train_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_train.csv', header=None)
test_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_test.csv', header=None)

# Map of original class labels
classes = {0: 'N', 1: 'S', 2: 'V', 3: 'F', 4: 'Q'}

# Add a new column for 'normal' vs 'not normal'
train_df['binary_label'] = train_df[187].apply(lambda x: 0 if x == 0 else 1)
test_df['binary_label'] = test_df[187].apply(lambda x: 0 if x == 0 else 1)

# Prepare the data
X_train = train_df.iloc[:, :187].values
y_train = train_df.iloc[:, 187].values
y_train_binary = train_df['binary_label'].values

X_test = test_df.iloc[:, :187].values
y_test = test_df.iloc[:, 187].values
y_test_binary = test_df['binary_label'].values

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_train_binary = torch.tensor(y_train_binary, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)
y_test_binary = torch.tensor(y_test_binary, dtype=torch.long)

# Create DataLoaders with both labels
batch_size = 1024

train_dataset = TensorDataset(X_train, y_train_binary, y_train)
test_dataset = TensorDataset(X_test, y_test_binary, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


class Inception1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Inception1D, self).__init__()
        self.branch1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)

        self.branch2 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=1),
            nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=1),
            nn.Conv1d(out_channels, out_channels, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
            nn.Conv1d(in_channels, out_channels, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        outputs = torch.cat([branch1, branch2, branch3, branch4], dim=1)
        return outputs

class InceptionNet1D(nn.Module):
    def __init__(self, num_classes=2):
        super(InceptionNet1D, self).__init__()
        self.initial_conv = nn.Conv1d(1, 32, kernel_size=3, stride=2, padding=1)

        self.inception1 = Inception1D(32, 32)
        self.inception2 = Inception1D(128, 64)
        self.inception3 = Inception1D(256, 128)
        self.inception4 = Inception1D(512, 256)

        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = F.relu(self.initial_conv(x))
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.inception3(x)
        x = self.inception4(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x



# Hyperparameters
lr = 0.001
num_epochs = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = InceptionNet1D().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for i, (inputs, binary_labels, original_labels) in enumerate(train_loader):
            inputs, binary_labels = inputs.unsqueeze(1).to(device), binary_labels.to(device)  # Add a channel dimension

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, binary_labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss/(i+1))
            pbar.update(1)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

model.eval()
def evaluate_model(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, binary_labels, original_labels in data_loader:
            inputs = inputs.unsqueeze(1).to(device)
            binary_labels = binary_labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, binary_labels)
            total_loss += loss.item() * inputs.size(0)

            # Compute accuracy
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == binary_labels).sum().item()
            total_samples += inputs.size(0)

    average_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    return average_loss, accuracy

# Evaluate the model
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
# Fonction pour extraire les caractéristiques
def extract_features(model, data_loader):
    features = []
    labels_list = []
    model.eval()
    with torch.no_grad():
        for inputs, binary_labels, original_labels in data_loader:
            inputs = inputs.unsqueeze(1).to(device)
            outputs = model(inputs)
            print(outputs.shape)
            features.append(outputs)
            labels_list.append(original_labels)
    features = torch.cat(features)
    labels_list = torch.cat(labels_list)
    return features, labels_list

# Extract features and labels from test data
test_features, test_labels = extract_features(model, test_loader)

# Calculate cosine similarity
distances = cosine_similarity(test_features.detach().cpu(), test_features.detach().cpu())
np.fill_diagonal(distances, float('-inf'))

# Functions for top-k accuracy
def find_max_indices(arr, k):
    if k > len(arr):
        raise ValueError("k cannot be greater than the size of the array")
    idx = np.argpartition(arr, -k)[-k:]
    sorted_idx = np.argsort(arr[idx])[::-1]
    return idx[sorted_idx]

def compute_top_k_accuracy(distances, all_ids, k):
    correct = 0
    for id_, elem in enumerate(distances, start=0):
        indices = find_max_indices(elem, k)
        candidates = [all_ids[indices[i]].item() for i in range(len(indices))]
        if all_ids[id_].item() in candidates:
            correct += 1
    accuracy = correct / len(distances)
    return accuracy

# Compute top-k accuracy
real_top_accuracies = []
for k in range(1, 6):
    accuracy = compute_top_k_accuracy(distances, test_labels, k)
    real_top_accuracies.append(accuracy)
    print(f"Top-{k} : {accuracy:.4f}")


Epoch 1/5:   0%|          | 0/86 [00:00<?, ?batch/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 32.55 GiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Epoch 1/5: 100%|██████████| 86/86 [00:29<00:00,  2.91batch/s, loss=0.427]


Epoch 1/5, Loss: 0.42747160965620085


Epoch 2/5: 100%|██████████| 86/86 [00:29<00:00,  2.92batch/s, loss=0.327]


Epoch 2/5, Loss: 0.32745449487553085


Epoch 3/5: 100%|██████████| 86/86 [00:30<00:00,  2.81batch/s, loss=0.293]


Epoch 3/5, Loss: 0.29316349743410597


Epoch 4/5: 100%|██████████| 86/86 [00:29<00:00,  2.89batch/s, loss=0.198]


Epoch 4/5, Loss: 0.19791635645683422


Epoch 5/5: 100%|██████████| 86/86 [00:30<00:00,  2.86batch/s, loss=0.168]


Epoch 5/5, Loss: 0.16751867783970611
Test Loss: 0.2213, Test Accuracy: 0.9276
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([388, 2])
Top-1 : 0.5463
Top-2 : 0.9126
Top-3 : 0.9308
Top-4 : 0.9405
Top-5 : 0.9470
